In [1]:
import os


from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
from OCC.Extend.DataExchange import read_step_file, export_shape_to_svg
from OCC.Display.WebGl.x3dom_renderer import X3DExporter

In [2]:
shp = read_step_file(os.path.join('..','assets', 'step', 'VentilatorAP203.step'))

In [3]:
x3dexp = X3DExporter(shp, None, None, True, (1,1,0), (0,0,0), 0, 0, (0,0,0), 2, 1)


In [4]:
x3dexp.compute()

In [5]:
len(x3dexp._triangle_sets)

1

In [6]:
x3dtriangles=x3dexp.to_x3dfile_string(0)

In [7]:
from OCC.Core.VrmlAPI import VrmlAPI_Writer

In [8]:
myWriter = VrmlAPI_Writer()

In [9]:
myWriter.Write(shp,'Vent.wrl')

True

In [10]:
from subprocess import *

In [11]:
run(['chmod', 'a+x', '../bin/tovrmlx3d'])

CompletedProcess(args=['chmod', 'a+x', '../bin/tovrmlx3d'], returncode=0)

In [12]:
x3difs=run(['../bin/tovrmlx3d','--encoding','xml','Vent.wrl'], stdout=PIPE)

In [13]:
x3domHTML = '''<script type='text/javascript' src='https://www.x3dom.org/download/dev/x3dom-full.debug.js'> </script> 
<link rel='stylesheet' type='text/css' href='https://www.x3dom.org/download/dev/x3dom.css'></link>'''

In [14]:
import xml.etree.ElementTree as ET

In [15]:
from IPython.display import HTML

In [16]:
x3domXML = ET.tostring(ET.XML(x3difs.stdout), encoding="unicode", short_empty_elements=False)

## x3dom  with VrmlAPI generated X3D

In [17]:
HTML(x3domHTML + x3domXML)

In [18]:
x3domX3Dtriangles = ET.tostring(ET.XML(x3dtriangles), encoding="unicode", short_empty_elements=False)

## x3dom with pythonocc generated X3D (triangles)

In [19]:
HTML(x3domHTML + x3domX3Dtriangles)

## Threejs baseed Jupyter renderer

In [20]:
my_renderer = JupyterRenderer()

In [21]:
my_renderer.DisplayShape(shp, render_edges=True, topo_level="Face", shape_color="#abdda4", update=True)

In [22]:
import x3d.x3d as x

x3d.py package loaded, have fun with X3D Graphics!


In [23]:
vp=x.Viewpoint(position=(-4.36117,-66.43609,65.70160),orientation=(0.99385,-0.09560,0.05589,0.74120),
               centerOfRotation=(0.00000,0.00000,0.00000),description="main view")

In [32]:
from OCC.Extend.TopologyUtils import TopologyExplorer

In [33]:
t = TopologyExplorer(shp)

In [97]:
textGeo=x.Text(fontStyle=x.FontStyle(family=['SANS'], 
                                      justify=["MIDDLE","MIDDLE"],
                                      size=5.0),
                string=["compounds: " + str(t.number_of_compounds()),
                        "shells: " + str(t.number_of_shells()),
                        "solids: " + str(t.number_of_solids()),
                        "faces: " + str(t.number_of_faces()),
                        "wires: " + str(t.number_of_wires()),
                        "vertices: " + str(t.number_of_vertices())
                       ],
              solid=False)
                

In [98]:
textShape=x.Shape(geometry=textGeo, appearance=x.Appearance(material=x.Material(diffuseColor=(0,0,1))))

In [25]:
import base64

In [26]:
stepInline='''<Inline namespacename='step' contentType='model/x3d+xml' url=' "data:model/x3d+xml;base64,{0}" ' '''.format(base64.b64encode(x3domX3Dtriangles.encode('utf-8')).decode('utf-8'))

In [118]:
scene=x.Scene(children=[vp, x.Transform(translation=(0, 0, 20), children=[textShape]), x.Group()])

In [114]:
scene.toXML()

"<Scene>\n  <Viewpoint description='main view' orientation='(0.99385, -0.0956, 0.05589, 0.7412)' position='(-4.36117, -66.43609, 65.7016)'/>\n  <Transform translation='(0, 0, -20)'>\n    <Shape>\n      <Appearance>\n        <Material diffuseColor='(0, 0, 1)'/>\n      </Appearance>\n      <Text string='['compounds: 0', 'shells: 1', 'solids: 1', 'faces: 305', 'wires: 309', 'vertices: 489']'>\n        <FontStyle family='['SANS']' justify='['MIDDLE', 'MIDDLE']' size='5.0'/>\n      </Text>\n    </Shape>\n  </Transform>\n  <Group/>\n</Scene>\n"

In [119]:
x3dDoc=x.X3D(head=x.head(),Scene=scene).toXML()

In [120]:
x3dDoc_noDecl=x3dDoc[x3dDoc.find('<X3D'):]\
.replace('(','').replace(')','')\
.replace("'[", ' " ').replace("]'", ' " ')\
.replace("'", 'xx"xx').replace(' " ', " ' ").replace('xx"xx', '"')\
.replace("<Group/>", stepInline)

      

In [121]:
HTML(x3domHTML + x3dDoc_noDecl)